In [1]:
from lxml import etree

In [2]:
class Defect:
    assemblyRevision = ''
    serialNumber = ''
    testerName = ''
    type = ''
    dataFolderName = ''
    indictmentType = ''
    repairOperator = ''
    repairStatus = ''
    designator = ''
    partId = ''
    pinId = ''

    def to_dict(self):
        res = {}
        res['assemblyRevision'] = self.assemblyRevision
        res['serialNumber'] = self.serialNumber
        res['testerName'] = self.testerName
        res['type'] = self.type
        res['dataFolderName'] = self.dataFolderName
        res['indictmentType'] = self.indictmentType
        res['repairOperator'] = self.repairOperator
        res['repairStatus'] = self.repairStatus
        res['designator'] = self.designator
        res['partId'] = self.partId
        res['pinId'] = self.pinId

        return res

In [104]:
def parse_file_for_defects(path, include_pin = True):
    root = etree.parse(path)
    ns = root.getroot().nsmap
    res = [] #Defect()
    serialNumber = root.getroot().find("ns1:BoardXML", ns).attrib.get('serialNumber')
    testerName = root.getroot().find("ns1:StationXML", ns).attrib.get('testerName')
    assemblyRevision = root.getroot().find("ns1:BoardXML", ns).attrib.get('assemblyRevision')
    dataFolderName = root.getroot().attrib.get('dataFolderName') 

    # TestXML = root.getroot().findall(".//ns1:TestXML", ns)
    TestXML = root.getroot().findall(".//ns1:TestXML", ns)
    for test in TestXML:
        name = test.attrib.get('name')
        IndictmentXML = test.findall(".//ns1:IndictmentXML", ns)
        PinXML = test.findall(".//ns1:PinXML", ns)
    
        for Indictment in IndictmentXML:
            tmp = Defect()
            tmp.assemblyRevision = assemblyRevision
            tmp.serialNumber = serialNumber
            tmp.testerName = testerName
            tmp.designator = name
            tmp.dataFolderName = dataFolderName
            tmp.type = 'IndictmentXML'
            tmp.indictmentType = Indictment.attrib.get('indictmentType')
            tmp.repairOperator = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairOperator')
            tmp.repairStatus = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairStatus')
            tmp.partId = Indictment.find("ns1:ComponentXML", ns).attrib.get('partId')
            res.append(tmp)

        if include_pin:
            partId = IndictmentXML[0].find("ns1:ComponentXML", ns).attrib.get('partId')
            repairOperator = IndictmentXML[0].find("ns1:RepairActionXML", ns).attrib.get('repairOperator')
            for pin in PinXML:
                repairStatus = pin.attrib.get('repairStatus')
                id = pin.attrib.get('id')
                PinIndictmentXML = pin.findall(".//ns1:PinIndictmentXML", ns)
                tmp.dataFolderName = dataFolderName

                for pinIndictment in PinIndictmentXML:
                    tmp = Defect()
                    tmp.assemblyRevision = assemblyRevision
                    tmp.serialNumber = serialNumber
                    tmp.testerName = testerName
                    tmp.designator = name
                    tmp.dataFolderName = dataFolderName
                    tmp.type = 'PinXML'
                    tmp.indictmentType = pinIndictment.attrib.get('indictmentType')
                    tmp.repairOperator = repairOperator
                    tmp.repairStatus = repairStatus
                    tmp.partId = partId
                    tmp.pinId = id
                    res.append(tmp)
        
    return res


In [4]:
from os import listdir
mypath = '/home/demo/Working/ML-services_PCBA/sample_data/all-vvts-xml-sample/'
all_files = listdir(mypath)

In [5]:
len(all_files), all_files[4]

(631,
 'Y1E3B44GAU0LU#rsrpci782009b#v810#D-SMT-AXI-71#3#1709671416000#PostRepair#1709671755000.xml')

In [6]:
all_defects = []

for file in all_files:
    tmp = parse_file_for_defects(mypath + file)
    for t in tmp:
        all_defects.append(t.to_dict())

In [7]:
len(all_defects)

107119

In [8]:
all_defects[100002]

{'assemblyRevision': 'RSRPCI782009B_B',
 'serialNumber': 'Y1E3B44GAU08O',
 'testerName': 'D-SMT-AOI-41',
 'type': 'PinXML',
 'dataFolderName': 'D-SMT-AOI-41[@$@]2024-03-01-18-39-53',
 'indictmentType': 'Missing',
 'repairOperator': None,
 'repairStatus': 'False Call',
 'designator': '2:j10',
 'partId': '10156545-123Q100LF',
 'pinId': '4'}

In [9]:
import pandas as pd

df = pd.DataFrame(all_defects)

In [10]:
df[df.repairStatus != 'False Call'].shape[0]

947

In [11]:
df[df.repairStatus == 'False Call'].shape[0] + df[df.repairStatus != 'False Call'].shape[0]

107119

In [12]:
df[df.repairStatus != 'False Call'].head

<bound method NDFrame.head of        assemblyRevision   serialNumber    testerName           type  \
207     RSRPCI782009B_B  Y1E3B44GAU0A0  D-SMT-AOI-41  IndictmentXML   
208     RSRPCI782009B_B  Y1E3B44GAU0A0  D-SMT-AOI-41         PinXML   
267       rsrpci782009b  Y1E3B44GAU0LU  D-SMT-AXI-71  IndictmentXML   
311       rsrpci782009b  Y1E3B44GAU0LU  D-SMT-AXI-71  IndictmentXML   
312       rsrpci782009b  Y1E3B44GAU0LU  D-SMT-AXI-71  IndictmentXML   
...                 ...            ...           ...            ...   
104730    rsrpci782009b  Y1E3B44GAU0LT  D-SMT-AXI-71  IndictmentXML   
104731    rsrpci782009b  Y1E3B44GAU0LT  D-SMT-AXI-71  IndictmentXML   
104732    rsrpci782009b  Y1E3B44GAU0LT  D-SMT-AXI-71  IndictmentXML   
104733    rsrpci782009b  Y1E3B44GAU0LT  D-SMT-AXI-71  IndictmentXML   
106828  RSRPCI782009B_B  Y1E3B44GAU08U  D-SMT-AOI-41  IndictmentXML   

                              dataFolderName indictmentType repairOperator  \
207     D-SMT-AOI-41[@$@]2024-03-01-17-

In [13]:
from IPython.display import display, HTML

display(df[df.repairStatus != 'False Call'])

,assemblyRevision,serialNumber,testerName,type,dataFolderName,indictmentType,repairOperator,repairStatus,designator,partId,pinId
207,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,Skew,Larisa_VVTS08,Repaired,2:c9,T520C107M010ATE045,
208,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,PinXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,BadJoint,None,Repaired,2:c9,T520C107M010ATE045,0
267,rsrpci782009b,Y1E3B44GAU0LU,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J10:A49,None,
311,rsrpci782009b,Y1E3B44GAU0LU,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J10:B41,None,
312,rsrpci782009b,Y1E3B44GAU0LU,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J10:B41,None,
...,...,...,...,...,...,...,...,...,...,...,...
104730,rsrpci782009b,Y1E3B44GAU0LT,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J11:B50,None,
104731,rsrpci782009b,Y1E3B44GAU0LT,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J11:B50,None,
104732,rsrpci782009b,Y1E3B44GAU0LT,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J11:B50,None,
104733,rsrpci782009b,Y1E3B44GAU0LT,D-SMT-AXI-71,IndictmentXML,None,Open,c_admin,Repaired,top:1:J11:B50,None,


In [14]:
df[(df.repairStatus == 'Repaired') & (df.dataFolderName.notna())].head()

,assemblyRevision,serialNumber,testerName,type,dataFolderName,indictmentType,repairOperator,repairStatus,designator,partId,pinId
207,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,Skew,Larisa_VVTS08,Repaired,2:c9,T520C107M010ATE045,
208,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,PinXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,BadJoint,None,Repaired,2:c9,T520C107M010ATE045,0
1499,MBDX86742001B_T,Y1MTB44B5Z04Q,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-04-14-58-49,Extra Part,Larisa_VVTS08,Repaired,1:c3947,CC0402KRX5R7BB105,
1500,MBDX86742001B_T,Y1MTB44B5Z04Q,D-SMT-AOI-41,PinXML,D-SMT-AOI-41[@$@]2024-03-04-14-58-49,Billboard,None,Repaired,1:c3947,CC0402KRX5R7BB105,0
1501,MBDX86742001B_T,Y1MTB44B5Z04Q,D-SMT-AOI-41,PinXML,D-SMT-AOI-41[@$@]2024-03-04-14-58-49,Coplanarity,None,Repaired,1:c3947,CC0402KRX5R7BB105,0


In [15]:
df_1file = df[df.dataFolderName == 'D-SMT-AOI-41[@$@]2024-03-01-09-58-01']
df_1file.describe()

,assemblyRevision,serialNumber,testerName,type,dataFolderName,indictmentType,repairOperator,repairStatus,designator,partId,pinId
count,343,343,343,343,343,343,107,343,343,343,343
unique,1,1,1,2,1,15,1,1,69,35,58
top,MBDX86742001B_T,Y1MTB44B5Z05I,D-SMT-AOI-41,PinXML,D-SMT-AOI-41[@$@]2024-03-01-09-58-01,Bridging,Svetlana_VVTS08,False Call,1:u399,GRM188C80G106KE47D,
freq,343,343,343,236,343,164,107,343,47,65,107


In [16]:
path_to_images = '/home/demo/Working/ML-services_PCBA/sample_data/2024.03/01/'
smpl = df_1file.sample(n=1)
smpl

,assemblyRevision,serialNumber,testerName,type,dataFolderName,indictmentType,repairOperator,repairStatus,designator,partId,pinId
47216,MBDX86742001B_T,Y1MTB44B5Z05I,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-09-58-01,Missing,Svetlana_VVTS08,False Call,1:c282,GRM188C80G106KE47D,


In [17]:
raw_des = smpl.designator.values[0]
des = raw_des.replace(':', '_')
dataFolderName = smpl.dataFolderName.values[0]
smpl_image_path = path_to_images + dataFolderName + '/'
print(smpl_image_path)

/home/demo/Working/ML-services_PCBA/sample_data/2024.03/01/D-SMT-AOI-41[@$@]2024-03-01-09-58-01/


In [18]:
raw_des = smpl.designator.values[0]
des = raw_des.replace(':', '_')
des

'1_c282'

In [19]:
from msgspec.json import decode
from msgspec import Struct

class ref_des(Struct):
    defect_image: str

class falsecall_refdes(Struct):
    ref_des: ref_des

class truecall_refdes(Struct):
    ref_des: ref_des

class call_summary(Struct):
    falsecall_refdes: falsecall_refdes
    truecall_refdes: truecall_refdes

class post_ticket(Struct):
    call_summary: call_summary

In [62]:
smpl_image_path = '/home/demo/Working/ML-services_PCBA/sample_data/2024.03/01/D-SMT-AOI-41[@$@]2024-03-01-17-17-01/'
path_json = smpl_image_path + 'json_data.json'

with open(path_json, "rb") as f:
    data = decode(f.read())

In [21]:
data['post_ticket']['call_summary']['falsecall_refdes'].items() #

dict_items([('3:u9', {'defect_name': ['RowOffset'], 'defect_image': 'E_4580_3-u9.jpg'}), ('3:u7', {'defect_name': ['Bridging'], 'defect_image': 'E_4580_3-u7.jpg'}), ('3:j11', {'defect_name': ['Left off', 'Right off'], 'defect_image': 'E_4580_3-j11.jpg'}), ('2:u10', {'defect_name': ['RowOffset'], 'defect_image': 'E_4580_2-u10.jpg'}), ('1:u10', {'defect_name': ['RowOffset'], 'defect_image': 'E_4580_1-u10.jpg'}), ('1:j9', {'defect_name': ['Missing'], 'defect_image': 'E_4580_1-j9.jpg'}), ('1:u9', {'defect_name': ['RowOffset'], 'defect_image': 'E_4580_1-u9.jpg'}), ('1:u1', {'defect_name': ['BadJoint'], 'defect_image': 'E_4580_1-u1.jpg'}), ('1:u11', {'defect_name': ['OCV Fail', 'Bridging'], 'defect_image': 'E_4580_1-u11.jpg'}), ('1:r7', {'defect_name': ['Extra Part'], 'defect_image': 'E_4580_1-r7.jpg'}), ('1:u2', {'defect_name': ['BadJoint'], 'defect_image': 'E_4580_1-u2.jpg'}), ('1:c9', {'defect_name': ['BadJoint'], 'defect_image': 'E_4580_1-c9.jpg'}), ('1:j10', {'defect_name': ['Left off',

In [72]:
df_js = df.from_dict(data['post_ticket']['call_summary']['falsecall_refdes'], orient='index')
df_js1 = df.from_dict(data['post_ticket']['call_summary']['truecall_refdes'], orient='index')

# df.index.name = 'designator'
# df_js['dataFolderName'] = 'D-SMT-AOI-41[@$@]2024-03-01-17-17-01'
# df_js.set_index('dataFolderName', append=True)
df_js.reset_index(inplace=True)
df_js.rename(columns={'index': 'designator'}, inplace=True)
df_js

,designator,defect_name,defect_image
0,3:u9,[RowOffset],E_4580_3-u9.jpg
1,3:u7,[Bridging],E_4580_3-u7.jpg
2,3:j11,"[Left off, Right off]",E_4580_3-j11.jpg
3,2:u10,[RowOffset],E_4580_2-u10.jpg
4,1:u10,[RowOffset],E_4580_1-u10.jpg
5,1:j9,[Missing],E_4580_1-j9.jpg
6,1:u9,[RowOffset],E_4580_1-u9.jpg
7,1:u1,[BadJoint],E_4580_1-u1.jpg
8,1:u11,"[OCV Fail, Bridging]",E_4580_1-u11.jpg
9,1:r7,[Extra Part],E_4580_1-r7.jpg


In [75]:
all_rows = data['post_ticket']['call_summary']['falsecall_refdes']
all_rows.update(data['post_ticket']['call_summary']['truecall_refdes'])

df_js = df.from_dict(all_rows, orient='index')
df_js.reset_index(inplace=True)
df_js.rename(columns={'index': 'designator'}, inplace=True)
df_js

,designator,defect_name,defect_image
0,3:u9,[RowOffset],E_4580_3-u9.jpg
1,3:u7,[Bridging],E_4580_3-u7.jpg
2,3:j11,"[Left off, Right off]",E_4580_3-j11.jpg
3,2:u10,[RowOffset],E_4580_2-u10.jpg
4,1:u10,[RowOffset],E_4580_1-u10.jpg
5,1:j9,[Missing],E_4580_1-j9.jpg
6,1:u9,[RowOffset],E_4580_1-u9.jpg
7,1:u1,[BadJoint],E_4580_1-u1.jpg
8,1:u11,"[OCV Fail, Bridging]",E_4580_1-u11.jpg
9,1:r7,[Extra Part],E_4580_1-r7.jpg


In [23]:
df.index.name

'designator'

In [78]:
def get_dataFolderName(path):
    root = etree.parse(path)
    dataFolderName = root.getroot().attrib.get('dataFolderName')
    return dataFolderName

Parse single file

In [27]:
files_path = '/home/demo/Working/ML-services_PCBA/sample_data/all-vvts-xml-sample'
file_name = 'Y1E3B44GAU0A0#RSRPCI782009B_B#V510#D-SMT-AOI-41#1#1709302598000#PostRepair#1709304186000.xml'
img_path = '/home/demo/Working/ML-services_PCBA/sample_data/2024.03/01'


In [110]:
def get_df_defects(file_path, include_pin):
    defects = []
    tmp = parse_file_for_defects(file_path, include_pin = include_pin)
    for t in tmp:
        defects.append(t.to_dict())

    df_defects = pd.DataFrame(defects)

    return df_defects


In [113]:
def get_img_df(img_folder):
    path_json = img_folder + 'json_data.json'

    with open(path_json, "rb") as f:
        data = decode(f.read())

    all_rows = data['post_ticket']['call_summary']['falsecall_refdes']
    all_rows.update(data['post_ticket']['call_summary']['truecall_refdes'])

    df_js = df.from_dict(all_rows, orient='index')
    df_js.reset_index(inplace=True)
    df_js.rename(columns={'index': 'designator'}, inplace=True)
    df_js['img_folder'] = img_folder
    
    return df_js

In [114]:
file_path = files_path + '/' + file_name
df_img = get_img_df(img_path + '/' + get_dataFolderName(file_path) + '/')
df_img

,designator,defect_name,defect_image,img_folder
0,3:u9,[RowOffset],E_4580_3-u9.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
1,3:u7,[Bridging],E_4580_3-u7.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
2,3:j11,"[Left off, Right off]",E_4580_3-j11.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
3,2:u10,[RowOffset],E_4580_2-u10.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
4,1:u10,[RowOffset],E_4580_1-u10.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
5,1:j9,[Missing],E_4580_1-j9.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
6,1:u9,[RowOffset],E_4580_1-u9.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
7,1:u1,[BadJoint],E_4580_1-u1.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
8,1:u11,"[OCV Fail, Bridging]",E_4580_1-u11.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
9,1:r7,[Extra Part],E_4580_1-r7.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...


In [111]:
file_path = files_path + '/' + file_name
df_defects = get_df_defects(file_path, False)
tmp = df_defects[df_defects.designator == '2:u11']
tmp

,assemblyRevision,serialNumber,testerName,type,dataFolderName,indictmentType,repairOperator,repairStatus,designator,partId,pinId
12,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,OCV Fail,Larisa_VVTS08,False Call,2:u11,SN74CBTLV3257PWR,
13,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,Bridging,Larisa_VVTS08,False Call,2:u11,SN74CBTLV3257PWR,


In [116]:
df_total = df_defects.merge(df_img, on='designator', how='inner')
df_total.head()

,assemblyRevision,serialNumber,testerName,type,dataFolderName,indictmentType,repairOperator,repairStatus,designator,partId,pinId,defect_name,defect_image,img_folder
0,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,RowOffset,Larisa_VVTS08,False Call,1:u10,PI3DBS12212AXUAEX,,[RowOffset],E_4580_1-u10.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
1,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,Missing,Larisa_VVTS08,False Call,1:j9,G40H2132112HR,,[Missing],E_4580_1-j9.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
2,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,RowOffset,Larisa_VVTS08,False Call,1:u9,PI3DBS12212AXUAEX,,[RowOffset],E_4580_1-u9.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
3,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,BadJoint,Larisa_VVTS08,False Call,1:u1,SN74LVC1G17DCKR,,[BadJoint],E_4580_1-u1.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...
4,RSRPCI782009B_B,Y1E3B44GAU0A0,D-SMT-AOI-41,IndictmentXML,D-SMT-AOI-41[@$@]2024-03-01-17-17-01,OCV Fail,Larisa_VVTS08,False Call,1:u11,SN74CBTLV3257PWR,,"[OCV Fail, Bridging]",E_4580_1-u11.jpg,/home/demo/Working/ML-services_PCBA/sample_dat...


In [2]:
fn = 'D-SMT-AOI-41[@$@]2024-03-01-09-53-26'
ffn = fn.split('-')[-4]
ffn

'01'